# Build End-to-End LLM project for a retail domain (t-shirts selling store).

Tech stack for this project
* UI: Streamlit
* LLM: Google Gemini LLM model
* Embeddings: Hugging Face
* Framework: Langchain

# Importing required modules

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain import hub
from dotenv import load_dotenv
import os
import getpass

# Insert API key

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Initializing Gemini AI model

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Establishing database connection

In [ ]:
db_user = "root"
db_password = "Kitsune#sql98"
db_host = "localhost"
db_name = "bank_data"

# Name of test databases
# "atliq_tshirts"
# "bank_data"

In [ ]:
connection_Uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
db = SQLDatabase.from_uri(connection_Uri)

In [ ]:
print(db.table_info)
print("--------")
print("--------")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")

# Core functionality

## SQL Query Generation

In [ ]:
"""
1. Pulls a specialized SQL generation prompt from LangChain Hub
2. Formats the prompt with database schema information
3. Sends the prompt to Gemini AI for query generation
4. Uses a second prompt to extract just the SQL from the response
"""

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

def write_query(question: str):
    """Generate SQL query from the user's question."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": question,
        }
    )
    response = llm.invoke(prompt.to_string())
    extraction_prompt = """
    Please extract the SQL query from the following text and return only the SQL query without any additional characters or formatting:

    {response}

    SQL Query:
    """
    # Format the prompt with the actual response
    prompt = extraction_prompt.format(response=response)
    # Invoke the language model with the prompt
    parsed_query = llm.invoke(prompt)
    return parsed_query.content

## Query Execution

In [ ]:
"""
This function creates a query execution tool and runs the generated SQL against your database, returning the raw results.
"""

def execute_query(query: str):
    """Execute the SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return execute_query_tool.invoke(query)

## Natural Language Answer Generation

In [ ]:
"""
This function takes the original question, generated SQL, and query results, then asks Gemini to formulate a human-friendly answer.
"""

def generate_answer(question: str, query: str, result: str):
    """Generate an answer using the query results."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {question}\n'
        f'SQL Query: {query}\n'
        f'SQL Result: {result}'
    )
    response = llm.invoke(prompt)
    return response.content

# Putting It All Together

In [ ]:
def model_output(input):
    query = write_query(input)
    result = execute_query(query) # type: ignore
    answer = generate_answer(input, query, result) # type: ignore

    print(f"Query: \n\n{query}")
    print("\n")
    print(answer)

In [ ]:
# Questions for the "altiq_tshirts" database
"""
"How many t-shirts do we have left for nike in extra small size and white color?"
"How much is the price of the inventory for all small size t-shirts?"
"If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
"If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"
"""

# question = string_value

In [ ]:
# Questions for the "bank_data" database
"""
"What is the total balance of all the accounts in the database?"
"Which account has the lowest balance and what is the client name?"
"Name of the client who has the highest credit card limit and what is the limit"
"Name of the client with most number of credit cards, how many and total limit."
"Name of the client with the second top number of credit cards, how many and total limit."
"""

question = "Proporciona el nombre y código de clientes con ahorros mayores a cero y su cantidad de tarjetas de crédito. Ordénalos de acuerdo con su número de tarjetas y muestra los resultados en formato tabular."

In [ ]:
model_output(question)